In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [8]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

### Numpy arrays to Pytorch tensors

In [10]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [11]:
y_train_tensor.shape

torch.Size([455])

## Defining the Model

In [15]:
class SimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)
    
    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred, y):
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)

        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1-y_pred)).mean()
        return loss


In [21]:
learning_rate = 0.1
epochs = 100

In [22]:
# Training Pipeline

model = SimpleNN(X_test_tensor)

for _ in range(epochs):

    # Forward Pass
    y_pred = model.forward(X_train_tensor)

    # Calculate Loss
    loss = model.loss_function(y_pred, y_train_tensor)

    # Backward Pass
    loss.backward()

    # Update Parameters
    with torch.no_grad():
        model.weights -= learning_rate*model.weights.grad
        model.bias -= learning_rate*model.bias.grad

    # Zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    print(f"Epoch {_ + 1} Loss: {loss}")

Epoch 1 Loss: 3.4018772059684013
Epoch 2 Loss: 3.2674653055256213
Epoch 3 Loss: 3.13112425298374
Epoch 4 Loss: 2.9954525248128787
Epoch 5 Loss: 2.859918458529482
Epoch 6 Loss: 2.7166372713652787
Epoch 7 Loss: 2.563807848699274
Epoch 8 Loss: 2.410787902081902
Epoch 9 Loss: 2.2628172372594
Epoch 10 Loss: 2.1186224649020677
Epoch 11 Loss: 1.9758780979504726
Epoch 12 Loss: 1.8370909305060061
Epoch 13 Loss: 1.7011559150488549
Epoch 14 Loss: 1.5709461485034277
Epoch 15 Loss: 1.448944417774929
Epoch 16 Loss: 1.3401817244736542
Epoch 17 Loss: 1.2429632206424404
Epoch 18 Loss: 1.1610536583632372
Epoch 19 Loss: 1.0938999098102629
Epoch 20 Loss: 1.0400702883228838
Epoch 21 Loss: 0.9974734236739433
Epoch 22 Loss: 0.9638017265145341
Epoch 23 Loss: 0.9369329606479179
Epoch 24 Loss: 0.9151236200228366
Epoch 25 Loss: 0.89703167944268
Epoch 26 Loss: 0.8816622373974892
Epoch 27 Loss: 0.8682956744248707
Epoch 28 Loss: 0.8564213361330197
Epoch 29 Loss: 0.8456824402775308
Epoch 30 Loss: 0.8358324710823782


In [23]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)

    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()

    print("Accuracy: ", accuracy)

Accuracy:  tensor(0.5880)
